jieba 的分词算法
主要有以下三种：

基于统计词典，构造前缀词典，基于前缀词典对句子进行切分，得到所有切分可能，根据切分位置，构造一个有向无环图（DAG）；
基于DAG图，采用动态规划计算最大概率路径（最有可能的分词结果），根据最大概率路径分词；
对于新词(词库中没有的词），采用有汉字成词能力的 HMM 模型进行切分。

In [1]:
import jieba

content = "现如今，机器学习和深度学习带动人工智能飞速的发展，并在图片处理、语音识别领域取得巨大成功。"
# 精确分词
# 精确分词：精确模式试图将句子最精确地切开，精确分词也是默认分词。
segs_1 = jieba.cut(content, cut_all=False)
print("/".join(segs_1))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/yw/k7z_d_3567g16ss9plk47x9w0000gn/T/jieba.cache
Loading model cost 0.715 seconds.
Prefix dict has been built succesfully.


现如今/，/机器/学习/和/深度/学习/带动/人工智能/飞速/的/发展/，/并/在/图片/处理/、/语音/识别/领域/取得/巨大成功/。


In [2]:
# 全模式

# 全模式分词：把句子中所有的可能是词语的都扫描出来，速度非常快，但不能解决歧义。

In [3]:
segs_3 = jieba.cut(content, cut_all=True)
print("/".join(segs_3))

现如今/如今///机器/学习/和/深度/学习/带动/动人/人工/人工智能/智能/飞速/的/发展///并/在/图片/处理///语音/识别/领域/取得/巨大/巨大成功/大成/成功//


In [4]:
# 搜索引擎模式

# 搜索引擎模式：在精确模式的基础上，对长词再次切分，提高召回率，适合用于搜索引擎分词。

In [5]:
segs_4 = jieba.cut_for_search(content)
print("/".join(segs_4))

如今/现如今/，/机器/学习/和/深度/学习/带动/人工/智能/人工智能/飞速/的/发展/，/并/在/图片/处理/、/语音/识别/领域/取得/巨大/大成/成功/巨大成功/。


In [6]:
segs_5 = jieba.lcut(content)
print(segs_5)

['现如今', '，', '机器', '学习', '和', '深度', '学习', '带动', '人工智能', '飞速', '的', '发展', '，', '并', '在', '图片', '处理', '、', '语音', '识别', '领域', '取得', '巨大成功', '。']


In [7]:
# jieba 可以很方便地获取中文词性，通过 jieba.posseg 模块实现词性标注。

In [8]:
import jieba.posseg as psg
print([(x.word,x.flag) for x in psg.lcut(content)])

[('现如今', 't'), ('，', 'x'), ('机器', 'n'), ('学习', 'v'), ('和', 'c'), ('深度', 'ns'), ('学习', 'v'), ('带动', 'v'), ('人工智能', 'n'), ('飞速', 'n'), ('的', 'uj'), ('发展', 'vn'), ('，', 'x'), ('并', 'c'), ('在', 'p'), ('图片', 'n'), ('处理', 'v'), ('、', 'x'), ('语音', 'n'), ('识别', 'v'), ('领域', 'n'), ('取得', 'v'), ('巨大成功', 'nr'), ('。', 'x')]


In [9]:
# 并行分词

# 并行分词原理为文本按行分隔后，分配到多个 Python 进程并行分词，最后归并结果。

In [10]:
jieba.enable_parallel(4) # 开启并行分词模式，参数为并行进程数 。
jieba.disable_parallel() # 关闭并行分词模式 。

In [11]:
# 获取分词结果中词列表的 top n

In [13]:
from collections import Counter
top5= Counter(segs_5).most_common(5)
print(top5)

[('，', 2), ('学习', 2), ('现如今', 1), ('机器', 1), ('和', 1)]


In [14]:
# 自定义添加词和字典

# 默认情况下，使用默认分词，是识别不出这句话中的“铁甲网”这个新词，这里使用用户字典提高分词准确性。

In [15]:
txt = "铁甲网是中国最大的工程机械交易平台。"
print(jieba.lcut(txt))

['铁甲', '网是', '中国', '最大', '的', '工程机械', '交易平台', '。']


In [16]:
# 手动添加词典
jieba.add_word("铁甲网")
print(jieba.lcut(txt))

['铁甲网', '是', '中国', '最大', '的', '工程机械', '交易平台', '。']


In [17]:
jieba.load_userdict('user_dict.txt')
print(jieba.lcut(txt))

['铁甲网', '是', '中国', '最大', '的', '工程机械', '交易平台', '。']


In [1]:
from pyhanlp import *
content = "现如今，机器学习和深度学习带动人工智能飞速的发展，并在图片处理、语音识别领域取得巨大成功。"
print(HanLP.segment(content))

[现如今/t, ，/w, 机器学习/gi, 和/cc, 深度/n, 学习/v, 带动/v, 人工智能/n, 飞速/d, 的/ude1, 发展/vn, ，/w, 并/cc, 在/p, 图片/n, 处理/vn, 、/w, 语音/n, 识别/vn, 领域/n, 取得/v, 巨大/a, 成功/a, 。/w]


In [2]:
txt = "铁甲网是中国最大的工程机械交易平台。"
print(HanLP.segment(txt))

[铁甲/n, 网/n, 是/vshi, 中国/ns, 最大/gm, 的/ude1, 工程机械/nz, 交易平台/nz, 。/w]


In [3]:
CustomDictionary.add("铁甲网")
CustomDictionary.insert("工程机械", "nz 1024")
CustomDictionary.add("交易平台", "nz 1024 n 1")
print(HanLP.segment(txt))

[铁甲网/nz, 是/vshi, 中国/ns, 最大/gm, 的/ude1, 工程机械/nz, 交易平台/nz, 。/w]
